In [19]:
# right video (brightfield) を少し下げて配置：まず1枚プレビュー → その設定で動画出力
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import ImageFont, ImageDraw, Image
import os

# === 入出力ファイルパス ===
video1_path = "Worm1_1_bleachcorrected_full_top90_mean_full_adjustedG297M1484_minpad.mp4"  # 蛍光（左）
video2_path = "Video2_fitted_resized_output.avi"                                           # 明視野（右）
output_path = "merged_resized_newTimestamp_v2.avi"
preview_path = "preview_merged_with_offset.png"
timestamp_csv_path = "Timestamps_worm1.csv"

# === オフセット設定（右＝明視野の縦位置）===
# 正の値: 下へ / 負の値: 上へ
BRIGHTFIELD_DY = 2   # ← ここを微調整してください（例: 8, 10, 12 など）

# === フォント（ラベル用） ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === アスペクト比調整関数 ===
def adjust_to_aspect(width, height, aspect_ratio=16/9):
    target_height = int(width / aspect_ratio)
    if target_height < height:
        target_height = height
        width = int(target_height * aspect_ratio)
    return width, target_height

# === 明視野を縦方向にオフセットしてキャンバスへ配置 ===
def place_with_vertical_offset(dst_h, src_img, dy):
    """
    dst_h: 左動画(蛍光)の高さに合わせるキャンバス高さ
    src_img: 右動画(明視野)フレーム（H2 x W2 x 3）
    dy: 縦のオフセット（+で下、-で上）
    戻り値: (dst_h x W2 x 3) のキャンバスに src を縦方向オフセットで貼り付けた画像
    """
    H2, W2 = src_img.shape[:2]
    canvas = np.zeros((dst_h, W2, 3), dtype=np.uint8)

    # 貼り付け範囲（クリップ済み）
    dst_y0 = max(0, dy)
    src_y0 = max(0, -dy)
    paste_h = min(H2 - src_y0, dst_h - dst_y0)
    if paste_h > 0:
        canvas[dst_y0:dst_y0 + paste_h, :, :] = src_img[src_y0:src_y0 + paste_h, :, :]
    return canvas

# === タイムスタンプCSV ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")
timestamps = df['Timestamp_T=0'].values

# === 動画読み込み ===
cap1 = cv2.VideoCapture(video1_path)
cap2 = cv2.VideoCapture(video2_path)
if not cap1.isOpened() or not cap2.isOpened():
    raise RuntimeError("❌ 動画の読み込みに失敗しました。")

# === FPS・フレーム数 ===
fps = 50.948  # 参照どおり
n_frames = int(min(cap1.get(cv2.CAP_PROP_FRAME_COUNT), cap2.get(cv2.CAP_PROP_FRAME_COUNT), len(timestamps)))

# === 解像度 ===
width1  = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
width2  = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))

# === セパレーターとキャンバス設定 ===
separator_width = 12
padding = 10
total_width  = width1 + separator_width + width2
total_height = max(height1, height2)
frame_width, frame_height = adjust_to_aspect(total_width + 2 * padding, total_height + 2 * padding)

# === 1枚プレビュー（最初のフレーム） ===
ret1, f1 = cap1.read()
ret2, f2 = cap2.read()
if not ret1:
    f1 = np.zeros((height1, width1, 3), dtype=np.uint8)
if not ret2:
    f2 = np.zeros((height2, width2, 3), dtype=np.uint8)

# 明視野（右）を下げて貼り付け
f2_canvas = place_with_vertical_offset(height1, f2, BRIGHTFIELD_DY)
separator = np.zeros((height1, separator_width, 3), dtype=np.uint8)
preview_combined = np.hstack((f1, separator, f2_canvas))
preview_combined = put_text_arial(preview_combined, f"T = {timestamps[0]:.3f} s", (10, 10), (255, 255, 255))

# 16:9キャンバス中央へ
padded = cv2.copyMakeBorder(preview_combined, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=(0,0,0))
canvas = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
y_off = (frame_height - padded.shape[0]) // 2
x_off = (frame_width  - padded.shape[1]) // 2
canvas[y_off:y_off + padded.shape[0], x_off:x_off + padded.shape[1]] = padded
cv2.imwrite(preview_path, canvas)
print(f"🖼 プレビューを書き出しました → {preview_path}")

# === キャプチャを先頭へ戻す（動画出力用） ===
cap1.set(cv2.CAP_PROP_POS_FRAMES, 0)
cap2.set(cv2.CAP_PROP_POS_FRAMES, 0)

# === 出力初期化（動画） ===
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
if not out.isOpened():
    raise RuntimeError("❌ 出力ファイルの作成に失敗しました")

print(f"▶ フレーム処理開始: {n_frames} フレーム（BRIGHTFIELD_DY={BRIGHTFIELD_DY}）")

# === 本番ループ ===
for i in tqdm(range(n_frames)):
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1:
        frame1 = np.zeros((height1, width1, 3), dtype=np.uint8)
    if not ret2:
        frame2 = np.zeros((height2, width2, 3), dtype=np.uint8)

    # 右(明視野)をオフセット配置
    frame2_canvas = place_with_vertical_offset(height1, frame2, BRIGHTFIELD_DY)

    # 結合
    separator = np.zeros((height1, separator_width, 3), dtype=np.uint8)
    combined = np.hstack((frame1, separator, frame2_canvas))

    # タイムスタンプ
    timestamp = timestamps[i]
    combined = put_text_arial(combined, f"T = {timestamp:.3f} s", (10, 10), (255, 255, 255))

    # パディング → 16:9 キャンバス中央配置
    padded_combined = cv2.copyMakeBorder(combined, padding, padding, padding, padding,
                                         cv2.BORDER_CONSTANT, value=(0, 0, 0))
    canvas = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
    y_offset = (frame_height - padded_combined.shape[0]) // 2
    x_offset = (frame_width  - padded_combined.shape[1]) // 2
    canvas[y_offset:y_offset + padded_combined.shape[0],
           x_offset:x_offset + padded_combined.shape[1]] = padded_combined

    out.write(canvas)

# === 終了処理 ===
cap1.release()
cap2.release()
out.release()
print("✅ 完了:", output_path)


🖼 プレビューを書き出しました → preview_merged_with_offset.png
▶ フレーム処理開始: 9995 フレーム（BRIGHTFIELD_DY=2）


100%|███████████████████████████████████████| 9995/9995 [02:23<00:00, 69.46it/s]

✅ 完了: merged_resized_newTimestamp_v2.avi


In [ ]:
#latest version?
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import ImageFont, ImageDraw, Image
import os

# === 入出力ファイルパス ===
video1_path = "Worm1_1_bleachcorrected_full_top90_mean_full_adjustedG297M1484_minpad.mp4" #蛍光
video2_path = "Video2_fitted_resized_output.avi" #明視野
output_path = "merged_resized_newTimestamp_v2.avi"
timestamp_csv_path = "Timestamps_worm1.csv"  # ← 追加：CSVのパス

# === Arialフォントのパス（Windows） ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === アスペクト比調整関数 ===
def adjust_to_aspect(width, height, aspect_ratio=16/9):
    target_height = int(width / aspect_ratio)
    if target_height < height:
        target_height = height
        width = int(target_height * aspect_ratio)
    return width, target_height

# === タイムスタンプCSV読み込み ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")

timestamps = df['Timestamp_T=0'].values  # NumPy配列に変換

# === 動画読み込み ===
cap1 = cv2.VideoCapture(video1_path)
cap2 = cv2.VideoCapture(video2_path)

if not cap1.isOpened() or not cap2.isOpened():
    raise RuntimeError("❌ 動画の読み込みに失敗しました。")

# === FPS・フレーム数取得 === 
# コメントアウトでFPSを変更;; 50.948:終了時刻から算出　50.943:各フレームの時間差平均から算出
fps = 50.948  
#fps = 50.943
n_frames = int(min(cap1.get(cv2.CAP_PROP_FRAME_COUNT), cap2.get(cv2.CAP_PROP_FRAME_COUNT), len(timestamps)))

# === 解像度取得 ===
width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))

# === セパレーターとキャンバス設定 ===
separator_width = 12
padding = 10
total_width = width1 + separator_width + width2
total_height = max(height1, height2)

# アスペクト比16:9に調整
frame_width, frame_height = adjust_to_aspect(total_width + 2 * padding, total_height + 2 * padding)

# === 出力初期化 ===
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
if not out.isOpened():
    raise RuntimeError("❌ 出力ファイルの作成に失敗しました")

print(f"▶ フレーム処理開始: {n_frames} フレーム")

# === フレーム処理ループ ===
for i in tqdm(range(n_frames)):
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1:
        frame1 = np.zeros((height1, width1, 3), dtype=np.uint8)
    if not ret2:
        frame2 = np.zeros((height2, width2, 3), dtype=np.uint8)

    # 明視野キャンバス（上揃え）
    frame2_canvas = np.zeros((height1, width2, 3), dtype=np.uint8)
    copy_height = min(height1, height2)
    frame2_canvas[0:copy_height, :, :] = frame2[0:copy_height, :, :]

    # セパレーター
    separator = np.zeros((height1, separator_width, 3), dtype=np.uint8)

    # 結合
    combined = np.hstack((frame1, separator, frame2_canvas))

    # タイムスタンプ表示（CSVから取得）
    timestamp = timestamps[i]
    combined = put_text_arial(combined, f"T = {timestamp:.3f} s", (10, 10), (255, 255, 255))

    # パディング追加
    padded_combined = cv2.copyMakeBorder(
        combined,
        top=padding, bottom=padding, left=padding, right=padding,
        borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )

    # キャンバス中央配置
    canvas = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
    y_offset = (frame_height - padded_combined.shape[0]) // 2
    x_offset = (frame_width - padded_combined.shape[1]) // 2
    canvas[y_offset:y_offset + padded_combined.shape[0],
           x_offset:x_offset + padded_combined.shape[1]] = padded_combined

    out.write(canvas)

# === 終了処理 ===
cap1.release()
cap2.release()
out.release()
print("✅ 完了:", output_path)


In [23]:
#スケールバーの小数点以下も反映させて描画するバージョンのmp4変換
import cv2
import numpy as np
import os
from PIL import ImageFont, ImageDraw, Image

# === フォント ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_I = ImageFont.truetype(font_path_I, font_size)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 入出力 ===
base_path = "/Users/yuusuke/Downloads/Videos/forVideo1-2"
input_path = os.path.join(base_path, "merged_resized_newTimestamp_v2.avi")
output_path = os.path.join(base_path, "merged_resized_newTimestamp_labeled_v3.mp4")

cap = cv2.VideoCapture(input_path)
if not cap.isOpened():
    raise RuntimeError("❌ 入力動画が開けません")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError("❌ 出力動画が作成できません")

# === ラベル・バー設定 ===
white = (255, 255, 255)
v_shift = 5

label_positions = {
    "xy": (18, 392 - v_shift),
    "xz": (18, 427 - v_shift),
    "zy": (577, 392 - v_shift),
}

scale_bar_start = (450, 410 - v_shift)  # (x, y)
scale_bar_length = 48.84                # 小数を含む
scale_bar_thickness = 3                 # 高さ（px）

# ---- 小数長を正確に反映するためのアルファ合成描画 ----
def _alpha_blend_strip(img, x, y, w, h, color, alpha):
    """img[y:y+h, x:x+w] に color を alpha でブレンド（x,y,w,h は整数・範囲内前提）"""
    if w <= 0 or h <= 0: 
        return
    H, W = img.shape[:2]
    x0, y0 = max(0, x), max(0, y)
    x1, y1 = min(W, x + w), min(H, y + h)
    if x1 <= x0 or y1 <= y0:
        return
    roi = img[y0:y1, x0:x1].astype(np.float32)
    overlay = np.empty_like(roi, dtype=np.float32)
    overlay[..., 0] = color[0]
    overlay[..., 1] = color[1]
    overlay[..., 2] = color[2]
    blended = (1.0 - alpha) * roi + alpha * overlay
    img[y0:y1, x0:x1] = blended.astype(np.uint8)

def draw_scale_bar_subpixel(img, start_xy, length_px, thickness_px, color=(255,255,255)):
    """
    横方向スケールバー（長さに小数を含む）を、端の1列をα合成で描くことで
    四捨五入せず視覚的に正確に反映する。
    """
    x0, y0 = start_xy
    H, W = img.shape[:2]
    if thickness_px <= 0 or length_px <= 0:
        return

    # 整数部分と小数部分に分解
    x_end_f = x0 + float(length_px)
    x_end_floor = int(np.floor(x_end_f))
    frac = x_end_f - x_end_floor  # 0 <= frac < 1

    # まず整数幅（full_w = floor(length)）を通常塗りつぶし
    full_w = x_end_floor - x0
    if full_w > 0:
        x_full0 = x0
        x_full1 = x0 + full_w - 1  # inclusive
        # クリップして描画
        x_full0_c = max(0, x_full0)
        x_full1_c = min(W - 1, x_full1)
        y1 = y0 + thickness_px - 1
        y0_c = max(0, y0)
        y1_c = min(H - 1, y1)
        if x_full1_c >= x_full0_c and y1_c >= y0_c:
            cv2.rectangle(img, (x_full0_c, y0_c), (x_full1_c, y1_c), color, thickness=-1)

    # 端の1列をα合成（小数分）
    if frac > 0.0:
        x_alpha = x_end_floor  # この1列に frac を掛ける
        # 範囲内ならブレンド
        if 0 <= x_alpha < W:
            _alpha_blend_strip(
                img,
                x_alpha, y0,
                1, thickness_px,
                color, float(frac)
            )

print(f"▶ ラベルとスケールバー描画中: {frame_count} フレーム")

# === ループ ===
for i in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break

    # ラベル
    for text, pos in label_positions.items():
        frame = put_text_arial_italic(frame, text, pos, white)

    # スケールバー（小数長をα合成で表現）
    draw_scale_bar_subpixel(frame, scale_bar_start, scale_bar_length, scale_bar_thickness, white)

    out.write(frame)

cap.release()
out.release()
print("✅ 完了:", output_path)


▶ ラベルとスケールバー描画中: 9995 フレーム
✅ 完了: /Users/yuusuke/Downloads/Videos/forVideo1-2/merged_resized_newTimestamp_labeled_v3.mp4


In [21]:
#上記コードの画像出力バージョン
# スケールバーの小数点以下も反映させて描画するバージョン + 最初の1枚を画像保存
import cv2
import numpy as np
import os
from PIL import ImageFont, ImageDraw, Image

# === フォント ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_I = ImageFont.truetype(font_path_I, font_size)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 入出力 ===
base_path = "/Users/yuusuke/Downloads/Videos/forVideo1-2"
input_path  = os.path.join(base_path, "merged_resized_newTimestamp_v2.avi")
output_path = os.path.join(base_path, "merged_resized_newTimestamp_labeled_v3.mp4")
preview_path = os.path.join(base_path, "merged_resized_newTimestamp_labeled_v3_firstframe.png")

# プレビューだけ欲しい場合は True に
PREVIEW_ONLY = False

cap = cv2.VideoCapture(input_path)
if not cap.isOpened():
    raise RuntimeError("❌ 入力動画が開けません")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = None
if not PREVIEW_ONLY:
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    if not out.isOpened():
        raise RuntimeError("❌ 出力動画が作成できません")

# === ラベル・バー設定 ===
white = (255, 255, 255)
v_shift = 5

label_positions = {
    "xy": (18, 392 - v_shift),
    "xz": (18, 427 - v_shift),
    "zy": (577, 392 - v_shift),
}

scale_bar_start = (450, 410 - v_shift)  # (x, y)
scale_bar_length = 48.84                # 小数を含む
scale_bar_thickness = 3                 # 高さ（px）

# ---- 小数長を正確に反映するためのアルファ合成描画 ----
def _alpha_blend_strip(img, x, y, w, h, color, alpha):
    """img[y:y+h, x:x+w] に color を alpha でブレンド（x,y,w,h は整数・範囲内前提）"""
    if w <= 0 or h <= 0:
        return
    H, W = img.shape[:2]
    x0, y0 = max(0, x), max(0, y)
    x1, y1 = min(W, x + w), min(H, y + h)
    if x1 <= x0 or y1 <= y0:
        return
    roi = img[y0:y1, x0:x1].astype(np.float32)
    overlay = np.empty_like(roi, dtype=np.float32)
    overlay[..., 0] = color[0]
    overlay[..., 1] = color[1]
    overlay[..., 2] = color[2]
    blended = (1.0 - alpha) * roi + alpha * overlay
    img[y0:y1, x0:x1] = blended.astype(np.uint8)

def draw_scale_bar_subpixel(img, start_xy, length_px, thickness_px, color=(255,255,255)):
    """
    横方向スケールバー（長さに小数を含む）を、端の1列をα合成で描くことで
    四捨五入せず視覚的に正確に反映する。
    """
    x0, y0 = start_xy
    H, W = img.shape[:2]
    if thickness_px <= 0 or length_px <= 0:
        return

    x_end_f = x0 + float(length_px)
    x_end_floor = int(np.floor(x_end_f))
    frac = x_end_f - x_end_floor  # 0 <= frac < 1

    # 整数部
    full_w = x_end_floor - x0
    if full_w > 0:
        x_full0_c = max(0, x0)
        x_full1_c = min(W - 1, x0 + full_w - 1)
        y0_c = max(0, y0)
        y1_c = min(H - 1, y0 + thickness_px - 1)
        if x_full1_c >= x_full0_c and y1_c >= y0_c:
            cv2.rectangle(img, (x_full0_c, y0_c), (x_full1_c, y1_c), color, thickness=-1)

    # 小数部（終端1列をαで）
    if frac > 0.0:
        x_alpha = x_end_floor
        if 0 <= x_alpha < W:
            _alpha_blend_strip(img, x_alpha, y0, 1, thickness_px, color, float(frac))

# === 最初の1枚のみ処理して保存 ===
ret, first_frame = cap.read()
if ret:
    # ラベル
    for text, pos in label_positions.items():
        first_frame = put_text_arial_italic(first_frame, text, pos, white)

    # スケールバー
    draw_scale_bar_subpixel(first_frame, scale_bar_start, scale_bar_length, scale_bar_thickness, white)

    # 保存（BGRのままでOK）
    ok = cv2.imwrite(preview_path, first_frame)
    if ok:
        print("🖼 1枚目を保存:", preview_path)
    else:
        print("⚠️ 1枚目の保存に失敗しました")

    # プレビューのみなら終了
    if PREVIEW_ONLY:
        cap.release()
        print("✅ プレビューのみ完了（動画は出力していません）")
        raise SystemExit(0)

    # 動画にも1枚目を書き込み
    out.write(first_frame)
else:
    print("⚠️ 1枚目の読み込みに失敗しました")

# === 2枚目以降は通常処理 ===
for _ in range(1, frame_count):
    ret, frame = cap.read()
    if not ret:
        break
    for text, pos in label_positions.items():
        frame = put_text_arial_italic(frame, text, pos, white)
    draw_scale_bar_subpixel(frame, scale_bar_start, scale_bar_length, scale_bar_thickness, white)
    out.write(frame)

cap.release()
if out is not None:
    out.release()
print("✅ 完了:", output_path)


🖼 1枚目を保存: /Users/yuusuke/Downloads/Videos/forVideo1-2/merged_resized_newTimestamp_labeled_v3_firstframe.png
✅ 完了: /Users/yuusuke/Downloads/Videos/forVideo1-2/merged_resized_newTimestamp_labeled_v3.mp4


In [10]:
import cv2
import numpy as np
import os
from PIL import ImageFont, ImageDraw, Image




# === Arial Italic フォントのパス（Windowsの場合）===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25


# イタリックフォント（例としてArial Italicを使う）
font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_size = 25
font_I = ImageFont.truetype(font_path_I, font_size)




def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    # OpenCVのBGR画像をPILのRGBに変換
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    # PIL画像をOpenCVのBGRに戻す
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === ファイルパス設定 ===
base_path = "/Users/yuusuke/Downloads/Videos/forVideo1-2"
input_path = os.path.join(base_path, "merged_resized_newTimestamp_v2.avi")  # 前処理済AVI
output_path = os.path.join(base_path, "merged_resized_newTimestamp_labeled_v3.mp4")  # 最終MP4

# === 入力動画読み込み ===
cap = cv2.VideoCapture(input_path)
if not cap.isOpened():
    raise RuntimeError("❌ 入力動画が開けません")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# === 出力動画設定（MP4, H.264形式）===
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError("❌ 出力動画が作成できません")

# === 描画パラメータ ===
white = (255, 255, 255)
v_shift = 5  # パディング・見た目の調整用

# ラベル位置
label_positions = {
    "xy": (18, 392 - v_shift),
    "xz": (18, 427 - v_shift),
    "zy": (577, 392 - v_shift),
}

# スケールバー（例：48.84ピクセル = 20 µm）
scale_bar_start = (450, 410 - v_shift)  # x, y位置
scale_bar_length = 48.84  # px
scale_bar_thickness = 3

print(f"▶ ラベルとスケールバー描画中: {frame_count} フレーム")

# === フレームごとの処理 ===
for i in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break

    # ラベル描画
    for text, pos in label_positions.items():
        frame = put_text_arial_italic(frame, text, pos, white)

    # スケールバー描画（横線）
    x0, y0 = scale_bar_start
    x1 = x0 + scale_bar_length
    cv2.rectangle(frame, (x0, y0), (x1, y0 + scale_bar_thickness), white, thickness=-1)

    # 書き出し
    out.write(frame)

# === 終了処理 ===
cap.release()
out.release()
print("✅ 完了:", output_path)


▶ ラベルとスケールバー描画中: 9995 フレーム
✅ 完了: /Users/yuusuke/Downloads/Videos/forVideo1-2/merged_resized_newTimestamp_labeled_v2.mp4
